In [1]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
import os
random.seed(5026)
target = np.sort([0] + random.sample(range(1, 69), 25))
num = []
for i in target:
    files = os.listdir(os.path.join("PIE", str(i)))
    n = len([j for j in files if j.endswith('.jpg')])
    num.append(n)
X = np.empty((sum(num), 32, 32))
y = np.empty((sum(num), 1))
for i, j in enumerate(target):
    for k, img_path in enumerate([f for f in os.listdir(os.path.join('PIE', str(j))) if f.endswith('.jpg')]):
        img_path = os.path.join('PIE', str(j), str(k+1)+'.jpg')
        X[sum(num[:i]) + k, :, :] = np.array(Image.open(img_path))
        y[sum(num[:i]) + k] = j
X_CMU = X[y.flatten() != 0, :, :]
y_CMU = y[y != 0]
X_self = X[y.flatten() == 0, :, :]
y_self = y[y == 0]
np.random.seed(42)
total_samples = len(X_CMU)
train_size = int(0.7 * total_samples)
train_indices = np.random.choice(total_samples, train_size, replace=False)
X_train_CMU = X_CMU[train_indices]
y_train_CMU = y_CMU[train_indices]
X_test_CMU = np.delete(X_CMU, train_indices, axis=0)
y_test_CMU = np.delete(y_CMU, train_indices, axis=0)
np.random.seed(42)
total_samples = len(X_self)
train_size = int(0.7 * total_samples)
train_indices = np.random.choice(total_samples, train_size, replace=False)
X_train_self = X_self[train_indices]
y_train_self = y_self[train_indices]
X_test_self = np.delete(X_self, train_indices, axis=0)
y_test_self = np.delete(y_self, train_indices, axis=0)
X_train = np.concatenate([X_train_CMU, X_train_self], axis = 0)
y_train = np.concatenate([y_train_CMU, y_train_self], axis = 0)
X_test = np.concatenate([X_test_CMU, X_test_self], axis = 0)
y_test = np.concatenate([y_test_CMU, y_test_self], axis = 0)

In [2]:
X_train_raw = X_train.reshape(X_train.shape[0], -1)
X_test_raw = X_test.reshape(X_test.shape[0], -1)
raw_face_images = [X_train_raw, X_test_raw]

X_train_reshape = X_train.reshape(X_train.shape[0], -1)
mean = np.mean(X_train_reshape, axis = 0)
std = np.std(X_train_reshape, axis = 0)
X_train_normalize = (X_train_reshape - mean) / std

X_train_CMU_reshape = X_train_CMU.reshape(X_train_CMU.shape[0], -1)
mean = np.mean(X_train_CMU_reshape, axis = 0)
std = np.std(X_train_CMU_reshape, axis = 0)
X_train_CMU_normalize = (X_train_CMU_reshape - mean) / std

X_train_self_reshape = X_train_self.reshape(X_train_self.shape[0], -1)
mean = np.mean(X_train_self_reshape, axis = 0)
std = np.std(X_train_self_reshape, axis = 0)
X_train_self_normalize = (X_train_self_reshape - mean) / std

X_test_reshape = X_test.reshape(X_test.shape[0], -1)
mean = np.mean(X_test_reshape, axis = 0)
std = np.std(X_test_reshape, axis = 0)
X_test_normalize = (X_test_reshape - mean) / std

X_test_CMU_reshape = X_test_CMU.reshape(X_test_CMU.shape[0], -1)
mean = np.mean(X_test_CMU_reshape, axis = 0)
std = np.std(X_test_CMU_reshape, axis = 0)
X_test_CMU_normalize = (X_test_CMU_reshape - mean) / std

X_test_self_reshape = X_test_self.reshape(X_test_self.shape[0], -1)
mean = np.mean(X_test_self_reshape, axis = 0)
std = np.std(X_test_self_reshape, axis = 0)
X_test_self_normalize = (X_test_self_reshape - mean) / std

In [3]:
C_values = [1e-2, 1e-1, 1]

In [4]:
from libsvm.svmutil import svm_train, svm_predict, svm_read_problem
import sys
original_stdout = sys.stdout
train_data_file = "train"
test_data_file = "test"
with open(train_data_file, "w") as train_file:
    for label, features in zip(y_train.flatten(), X_train_raw):
        train_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
with open(test_data_file, "w") as test_file:
    for label, features in zip(y_test.flatten(), X_test_raw):
        test_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
for C in C_values:
    l_train, X_train = svm_read_problem(train_data_file)
    l_test, X_test = svm_read_problem(test_data_file)
    svm_param = f'-t 0 -c {C} -q'
    sys.stdout = open(os.devnull, "w")
    svm_model = svm_train(l_train, X_train, svm_param)
    sys.stdout = original_stdout
    p_label, _, _ = svm_predict(l_test, X_test, svm_model)
    correct_predictions = np.sum(np.array(p_label) == y_test)
    total_samples = len(y_test)
    test_accuracy = correct_predictions / total_samples
    print(f"C = {C}")
    print("----------------")
os.remove(train_data_file)
os.remove(test_data_file)

Accuracy = 98.5133% (1259/1278) (classification)
C = 0.01
----------------
Accuracy = 98.5133% (1259/1278) (classification)
C = 0.1
----------------
Accuracy = 98.5133% (1259/1278) (classification)
C = 1
----------------


In [5]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean
        cov_matrix = np.cov(X_centered, rowvar = False)
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        sorted_indices = np.argsort(eigenvalues)[::-1]
        self.components = eigenvectors[:, sorted_indices[:self.n_components]]

    def transform(self, X):
        X_centered = X - self.mean
        return np.dot(X_centered, self.components)

In [6]:
pca_80 = PCA(n_components = 80)
pca_80.fit(X_train_normalize)
X_train_80 = pca_80.transform(X_train_normalize)
X_test_80 = pca_80.transform(X_test_normalize)

pca_200 = PCA(n_components = 200)
pca_200.fit(X_train_normalize)
X_train_200 = pca_200.transform(X_train_normalize)
X_test_200 = pca_200.transform(X_test_normalize)

In [7]:
original_stdout = sys.stdout
train_data_file = "train"
test_data_file = "test"
with open(train_data_file, "w") as train_file:
    for label, features in zip(y_train.flatten(), X_train_80):
        train_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
with open(test_data_file, "w") as test_file:
    for label, features in zip(y_test.flatten(), X_test_80):
        test_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
for C in C_values:
    l_train, X_train = svm_read_problem(train_data_file)
    l_test, X_test = svm_read_problem(test_data_file)
    svm_param = f'-t 0 -c {C} -q'
    sys.stdout = open(os.devnull, "w")
    svm_model = svm_train(l_train, X_train, svm_param)
    sys.stdout = original_stdout
    p_label, _, _ = svm_predict(l_test, X_test, svm_model)
    correct_predictions = np.sum(np.array(p_label) == l_test)
    total_samples = len(y_test)
    test_accuracy = correct_predictions / total_samples
    print(f"C = {C}")
    print("----------------")
os.remove(train_data_file)
os.remove(test_data_file)

Accuracy = 97.4961% (1246/1278) (classification)
C = 0.01
----------------
Accuracy = 97.7308% (1249/1278) (classification)
C = 0.1
----------------
Accuracy = 97.7308% (1249/1278) (classification)
C = 1
----------------


In [8]:
original_stdout = sys.stdout
train_data_file = "train"
test_data_file = "test"
with open(train_data_file, 'w') as train_file:
    for label, features in zip(y_train.flatten(), X_train_200):
        train_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
with open(test_data_file, 'w') as test_file:
    for label, features in zip(y_test.flatten(), X_test_200):
        test_file.write(f"{label} {' '.join([f'{i + 1}:{value}' for i, value in enumerate(features)])}\n")
for C in C_values:
    l_train, X_train = svm_read_problem(train_data_file)
    l_test, X_test = svm_read_problem(test_data_file)
    svm_param = f"-t 0 -c {C} -q"
    sys.stdout = open(os.devnull, "w")
    svm_model = svm_train(l_train, X_train, svm_param)
    sys.stdout = original_stdout
    p_label, _, _ = svm_predict(l_test, X_test, svm_model)
    correct_predictions = np.sum(np.array(p_label) == l_test)
    total_samples = len(y_test)
    test_accuracy = correct_predictions / total_samples
    print(f"C = {C}")
    print("----------------")
os.remove(train_data_file)
os.remove(test_data_file)

Accuracy = 98.0438% (1253/1278) (classification)
C = 0.01
----------------
Accuracy = 98.3568% (1257/1278) (classification)
C = 0.1
----------------
Accuracy = 98.3568% (1257/1278) (classification)
C = 1
----------------
